In [1]:
from dataset_io import *
from color_channels import equalize_hist
import os
import cv2
import numpy as np
import random as ra
from tqdm import tqdm
from profiler import *
import random
from PIL import Image, ImageCms
from joblib import load

In [2]:
def imread_resized(scene_name, filename, label_resolution):
    filepath=get_filepath(scene_name, filename)
    image = cv2.imread(filepath)
    sh=image.shape
    sh=(int(sh[1]/3), int(sh[0]/3))
    return cv2.resize(image, sh)

In [3]:
def convert_to_gradient_channels(img):
    channels=[]
    
    img = cv2.UMat(img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY )
    channels.append(cv2.UMat(gray))
    
    gray = cv2.GaussianBlur(gray, (3,3), sigmaX=0, sigmaY=0)
    channels.append(cv2.UMat(gray))
    
    gray = cv2.equalizeHist(gray) #experimental
    gradX = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3, scale=1 )
    gradY = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3, scale=1 )
    mag, ori = cv2.cartToPolar(gradX, gradY, angleInDegrees=True)

    ori_ranges = [(0,60), (60,120), (120,180), (180,240), (240,300), (300,360)]
    oris=[cv2.inRange(ori,l,r) for (l,r) in ori_ranges]
    mags= [cv2.copyTo(mag, ori_x) for ori_x in oris]
    channels.extend([gray,mag]+mags)
    channels=[cv2.UMat.get(ch) for ch in channels]
    print(np.array(channels).shape)
    #channels=np.moveaxis(channels, 0, -1)
    return channels

In [4]:
def pil_bgr_to_cmyk(bgr):
    img = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    img = ImageCms.profileToProfile(img, 'color_profiles/AdobeRGB1998.icc', 'color_profiles/USWebUncoated.icc', outputMode='CMYK')
#     img = img.convert('CMYK')
    return np.asarray(img)

def to_color_channels(bgr):
    channels=[]
    
    bgr=equalize_hist(bgr)
    print(np.array(bgr).shape)
    
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)  
    print("hsv",hsv.shape)
    channels.append(hsv[:,:,0])
    channels.append(hsv[:,:,1])
    print(len(channels))
    
    cmyk = pil_bgr_to_cmyk(bgr)
    print("cmyk",cmyk.shape)
    channels.append(cmyk[:,:,0])
    channels.append(cmyk[:,:,1])
    channels.append(cmyk[:,:,2])
    print(len(channels))
    
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
    print("lab",lab.shape)
#     channels.append(lab[:,:,1])
    channels.append(lab[:,:,2])
    print(len(channels))
    channels=[cv2.equalizeHist(ch) for ch in channels]
    
    return channels

In [5]:
KERNEL_W_RATIO=7.0/640.0

def binarize(channels):
    channels=np.array(channels)
    img=np.moveaxis(channels, 0, -1)
    
    os.chdir(PROJECT_PATH)
    color_clfs=[load('naive_bayes_clf_{}.joblib'.format(color)) for color in COLORS]
    color_clf_map=dict(zip(COLORS, color_clfs))
    
    s0, s1, s2 = img.shape
    flat_img = np.reshape(img, (s0*s1, s2))
    flat_binaries=[clf.predict(flat_img) for clf in color_clfs]
    img_binaries=[np.reshape(flat, (s0, s1)) for flat in flat_binaries]
    return img_binaries

def open_bin(img):
    h,w=img.shape
    k=int(w*KERNEL_W_RATIO)
    kernel=(k,k)
    kernel = np.ones(kernel, np.uint8) 
    img_erosion = cv2.erode(img, kernel, iterations=1)
    img_dilation = cv2.dilate(img_erosion, kernel, iterations=1)
    return img_dilation

def close_bin(img):
    h,w=img.shape
    k=int(w*KERNEL_W_RATIO)
    kernel=(k,k)
    print(kernel)
    kernel = np.ones(kernel, np.uint8) 
    img_dilation = cv2.dilate(img, kernel, iterations=1)
    img_erosion = cv2.erode(img_dilation, kernel, iterations=1)
    return img_erosion

def morpho(img):
    h,w=img.shape
    k=int(w*KERNEL_W_RATIO)
    kernel=(k,k)
    return np.bool_(close_bin(open_bin(np.uint8(img), kernel), kernel))

In [17]:
img = imread_resized("wooden_folia_6", "0001.jpg", None)
print(img.shape)
cv2.imwrite("visualized/img.jpg", img)

(240, 426, 3)


True

In [7]:
channels=convert_to_gradient_channels(img)

for i in range(len(channels)):
    cv2.imwrite("visualized/channel_{}.jpg".format(i), channels[i])

(10, 240, 426)


In [43]:
channels=to_color_channels(img)

# for i in range(len(channels)):
#     cv2.imwrite("visualized/color_channel_{}.jpg".format(i), channels[i])

(160, 213, 3)
hsv (160, 213, 3)
2
cmyk (160, 213, 4)
5
lab (160, 213, 3)
6


In [46]:
binary_imgs = binarize(channels)
print(binary_imgs[0].shape)
for i in range(len(binary_imgs)):
    img=np.uint8(binary_imgs[i])*255
    print(img)
    cv2.imwrite("visualized/binary_img_{}.jpg".format(i), img)
    closed=close_bin(img)
    cv2.imwrite("visualized/closed_img_{}.jpg".format(i), closed)
    opened=open_bin(closed)
    cv2.imwrite("visualized/opened_img_{}.jpg".format(i), opened)


(160, 213)
[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0 255]
 [  0   0   0 ... 255   0   0]
 [  0   0 255 ...   0   0   0]]
(2, 2)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(2, 2)
[[  0   0   0 ... 255   0 255]
 [255   0   0 ... 255   0   0]
 [  0   0   0 ... 255   0 255]
 ...
 [  0   0   0 ... 255   0   0]
 [  0   0 255 ...   0   0   0]
 [  0   0   0 ... 255 255 255]]
(2, 2)


In [18]:
k=3
h=18*k
w=100*k
window=img[0:h, 0:w]
cv2.imwrite("visualized/window.jpg", window)
cv2.rectangle(img, (0,0), (w,h), (0, 255, 0), 4)
cv2.imwrite("visualized/img_window.jpg", img)

True